## Get BaseClassifiers statistics

In [1]:
import torch
import matplotlib.pyplot as plt
import os
from sklearn.metrics import accuracy_score
import joblib
import sys
import numpy as np
import json
import time
from sklearn.metrics import (
    accuracy_score, recall_score, f1_score, precision_score, roc_auc_score,
    top_k_accuracy_score
)


CLIP_FEATURES_DIR = "clip_features"
VAL = CLIP_FEATURES_DIR + "/val_features.pt"
SCALER = "scaler_model.joblib"
PCA = "pca_model.joblib"
LIME = "top_k_lime_indices.joblib"

In [2]:
def load_features(file_path):
    data = torch.load(file_path)
    return data["image_features"], data["text_features"], data["filenames"], data["labels"]


# Load train and validation features
val_img_features, val_txt_features, _, val_labels = load_features(VAL)

In [3]:
# Combine image and text features for training
X_val = torch.cat((val_img_features, val_txt_features), dim=1)

# Flatten features into a 2D matrix (samples x features)
X_val = X_val.view(X_val.size(0), -1).numpy()

# Print the shape of the features
print(f"X_val shape: {X_val.shape}")

# Convert labels to NumPy arrays
y_val = val_labels.numpy()  

# Load scaler and PCA models
scaler = joblib.load(SCALER)
pca = joblib.load(PCA)
lime = joblib.load(LIME)

# Scale and transform the features
X_val_scaled = scaler.transform(X_val)
X_val_pca = pca.transform(X_val_scaled)
X_val_lime = X_val_scaled[:, lime]


print(f"X_val_pca shape: {X_val_pca.shape}")
print(f"X_val_lime shape: {X_val_lime.shape}")

X_val shape: (1985, 1024)
X_val_pca shape: (1985, 563)
X_val_lime shape: (1985, 250)


In [ ]:
from classifiers import (
    SVMClassifier, RBFClassifier, RandomForestClassifier, NaiveBayesClassifier, 
    LogisticRegressionClassifier, LDAClassifier, KNNClassifier, DecisionTreeClassifier,
    AdaBoostClassifier, GBMClassifier, XGBoostClassifier
)

# Initialize classifiers
classifiers = {
    "SVM": SVMClassifier(),
    "RBF": RBFClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Naive Bayes": NaiveBayesClassifier(),
    "Logistic Regression": LogisticRegressionClassifier(),
    "LDA": LDAClassifier(),
    "KNN": KNNClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "AdaBoost": AdaBoostClassifier(),
    "Gradient Boosting": GBMClassifier(),
    "XGBoost": XGBoostClassifier()
}

results = {}
models = ["models_pca", "models_lime"]

# Iterate over classifiers
for model in models:
    results[model] = {}
    for name, classifier in classifiers.items():
        print(f"\nEvaluating {name}...")
        try:
            classifier.load(model_dir=model)
            label_encoder = classifier.label_encoder
            all_labels = label_encoder.classes_

            # Time classification
            start_time = time.time()
            y_pred = classifier.classify(X_val_pca)
            elapsed_time = time.time() - start_time

            # Check if classify_proba is available
            if hasattr(classifier, "classify_proba"):
                try:
                    y_proba = classifier.classify_proba(X_val_pca)
                    top5_acc = top_k_accuracy_score(y_val, y_proba, k=5, labels=np.arange(1, 91))

                    # AUC-ROC computation — assumes multi-class OVR
                    auc_roc = roc_auc_score(y_val, y_proba, multi_class='ovr', average='weighted', labels=np.arange(1, 91))
                except Exception as e:
                    print(f"Probability-based metrics not available for {name}: {e}")
                    top5_acc = None
                    auc_roc = None
            else:
                top5_acc = None
                auc_roc = None

            acc = accuracy_score(y_val, y_pred)
            recall = recall_score(y_val, y_pred, average='weighted')
            f1 = f1_score(y_val, y_pred, average='weighted')
            precision = precision_score(y_val, y_pred, average='weighted')  # ← New metric

            results[model][name] = {
                "top-5 accuracy": top5_acc,
                "accuracy": acc,
                "recall": recall,
                "precision": precision,                # ← New metric
                "f1-score": f1,
                "auc-roc": auc_roc,                    # ← New metric
                "inference_time_sec": elapsed_time
            }

            print(f"{name} -> Accuracy: {acc:.4f}, F1: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, Time: {elapsed_time:.4f}s")
            if top5_acc is not None:
                print(f"Top-5 Accuracy: {top5_acc:.4f}")
            if auc_roc is not None:
                print(f"AUC-ROC: {auc_roc:.4f}")

        except Exception as e:
            print(f"Failed to evaluate {name}: {e}")
            results[model][name] = {
                "top-5 accuracy": None,
                "accuracy": None,
                "recall": None,
                "precision": None,
                "f1-score": None,
                "auc-roc": None,
                "inference_time_sec": None,
                "error": str(e)
            }
    


Evaluating SVM...
Loaded model from: models_pca/SVM.joblib
Loaded label encoder from: models_pca/label_encoder.joblib
Probability-based metrics not available for SVM: Number of classes in y_true not equal to the number of columns in 'y_score'
SVM -> Accuracy: 0.6247, F1: 0.6076, Precision: 0.6079, Recall: 0.6247, Time: 2.9883s

Evaluating RBF...
Loaded model from: models_pca/RBF.joblib
Loaded label encoder from: models_pca/label_encoder.joblib


/home/tiago/thesis/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Probability-based metrics not available for RBF: Number of classes in y_true not equal to the number of columns in 'y_score'
RBF -> Accuracy: 0.6448, F1: 0.5954, Precision: 0.5895, Recall: 0.6448, Time: 5.0745s

Evaluating Random Forest...
Loaded model from: models_pca/RandomForest.joblib
Loaded label encoder from: models_pca/label_encoder.joblib


/home/tiago/thesis/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Probability-based metrics not available for Random Forest: Number of classes in y_true not equal to the number of columns in 'y_score'
Random Forest -> Accuracy: 0.4408, F1: 0.3020, Precision: 0.3116, Recall: 0.4408, Time: 0.0518s

Evaluating Naive Bayes...
Loaded model from: models_pca/NaiveBayes.joblib
Loaded label encoder from: models_pca/label_encoder.joblib


...

/home/tiago/thesis/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/tiago/thesis/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/tiago/thesis/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Probability-based metrics not available for KNN: Number of classes in y_true not equal to the number of columns in 'y_score'
KNN -> Accuracy: 0.6151, F1: 0.5831, Precision: 0.5743, Recall: 0.6151, Time: 0.1548s

Evaluating Decision Tree...
Loaded model from: models_lime/DecisionTree.joblib
Loaded label encoder from: models_lime/label_encoder.joblib
Probability-based metrics not available for Decision Tree: Number of classes in y_true not equal to the number of columns in 'y_score'
Decision Tree -> Accuracy: 0.4126, F1: 0.4122, Precision: 0.4184, Recall: 0.4126, Time: 0.0011s

Evaluating AdaBoost...
Loaded model from: models_lime/AdaBoost.joblib
Loaded label encoder from: models_lime/label_encoder.joblib
Probability-based metrics not available for AdaBoost: Number of classes in y_true not equal to the number of columns in 'y_score'
AdaBoost -> Accuracy: 0.3612, F1: 0.1937, Precision: 0.1386, Recall: 0.3612, Time: 0.0443s

Evaluating Gradient Boosting...


/home/tiago/thesis/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/tiago/thesis/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/tiago/thesis/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Loaded model from: models_lime/GBM.joblib
Loaded label encoder from: models_lime/label_encoder.joblib
Probability-based metrics not available for Gradient Boosting: Number of classes in y_true not equal to the number of columns in 'y_score'
Gradient Boosting -> Accuracy: 0.4690, F1: 0.4449, Precision: 0.4437, Recall: 0.4690, Time: 0.0504s

Evaluating XGBoost...
Loaded model from: models_lime/XGBoost.joblib
Loaded label encoder from: models_lime/label_encoder.joblib
Probability-based metrics not available for XGBoost: Number of classes in y_true not equal to the number of columns in 'y_score'
XGBoost -> Accuracy: 0.6025, F1: 0.5556, Precision: 0.5600, Recall: 0.6025, Time: 0.0298s


/home/tiago/thesis/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/tiago/thesis/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
with open("classifier_results_new.json", "w") as f:
    json.dump(results, f, indent=4)

print("Results saved to classifier_results.json")

Results saved to classifier_results_new.json
